In [1]:
import os
import pandas as pd
import numpy as np
import re
from pathlib import Path 

In [9]:
DATA_DAILY = Path("data/daily")
DATA_HOT_SCORE = Path("data/hotscore")
OUTPUT_DIR = Path("output/logistic_regression")

for p in (DATA_DAILY, DATA_HOT_SCORE, OUTPUT_DIR):
    p.mkdir(parents=True, exist_ok=True)

In [10]:
def latest_file_in_directory(directory=DATA_HOT_SCORE):
    latest_file = max(
        f for f in os.listdir(directory)
        if f.startswith("hotscore_") and f.endswith(".csv")
    )
    return latest_file


In [50]:
latest_file = latest_file_in_directory(DATA_HOT_SCORE)
df = pd.read_csv(os.path.join(DATA_HOT_SCORE, latest_file))
df.shape

(44222, 8)

In [52]:
display(df.head())

,symbol,date,HotScore,regularMarketPrice,regularMarketChangePercent,VolumeSpike,averageDailyVolume3Month,marketCap
0,AA,2025-11-26 20:46:26,0.794401,41.845,6.747450,0.940394,6727448.0,1.083635e+10
1,AAUC,2025-11-26 20:46:26,0.846094,19.180,9.788214,1.052893,342331.0,2.378320e+09
2,ALAB,2025-11-26 20:46:26,0.773307,156.160,7.860205,0.638383,6266829.0,2.637599e+10
3,ANF,2025-11-26 20:46:26,0.905599,96.205,6.610155,1.883490,2070173.0,4.583525e+09
4,ARWR,2025-11-26 20:46:26,0.956380,58.675,25.400724,2.501594,2311350.0,8.112262e+09


In [53]:
df = df.copy()

# Target: HotScore > 0.85
df["is_hot"] = (df["HotScore"] > 0.85).astype(int)

# Clip values to avoid log(0) or negative values
df["marketCap"] = df["marketCap"].clip(lower=1)
df["averageDailyVolume3Month"] = df["averageDailyVolume3Month"].clip(lower=1)

# Log-transform
df["log_marketCap"] = np.log(df["marketCap"])
df["log_volume"] = np.log(df["averageDailyVolume3Month"])

# Features (exclude symbol, date, HotScore)
features = [
    "regularMarketPrice",
    "regularMarketChangePercent",
    "VolumeSpike",
    "log_marketCap",
    "log_volume"
]

X = df[features]
y = df["is_hot"]

# Keep symbol and date for final table
symbol_date = df[["symbol", "date"]]

In [54]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, sym_train, sym_test = train_test_split(
    X, y, symbol_date, test_size=0.25, random_state=42, stratify=y
)

In [55]:
has_inf = np.isinf(X_train.values).any()
has_nan = np.isnan(X_train.values).any()
needs_cleaning = has_inf or has_nan
print("Need cleaning?", needs_cleaning)

if needs_cleaning:
    print("Cleaning X_train and X_test...")
    X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Keep indexes aligned
    keep_train_idx = X_train.dropna().index
    X_train = X_train.loc[keep_train_idx]
    y_train = y_train.loc[keep_train_idx]

    keep_test_idx = X_test.dropna().index
    X_test = X_test.loc[keep_test_idx]
    y_test = y_test.loc[keep_test_idx]
    sym_test = sym_test.loc[keep_test_idx]


Need cleaning? True
Cleaning X_train and X_test...


In [56]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, classification_report

log_model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    solver="lbfgs"
)
log_model.fit(X_train_scaled, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [58]:
y_proba = log_model.predict_proba(X_test_scaled)[:, 1]
y_pred = log_model.predict(X_test_scaled)

print("ROC AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))

ROC AUC: 0.8618396812280005
              precision    recall  f1-score   support

           0       0.89      0.81      0.85      7966
           1       0.59      0.75      0.66      3020

    accuracy                           0.79     10986
   macro avg       0.74      0.78      0.75     10986
weighted avg       0.81      0.79      0.80     10986



In [59]:
coef_df = pd.DataFrame({
    "feature": features,
    "coefficient": log_model.coef_[0]
}).sort_values("coefficient", ascending=False)

display(coef_df)

,feature,coefficient
1,regularMarketChangePercent,2.497742
2,VolumeSpike,2.093978
3,log_marketCap,0.711709
0,regularMarketPrice,0.013504
4,log_volume,-0.988001


In [61]:
results = X_test.copy()
results["symbol"] = sym_test["symbol"].values
results["date"] = sym_test["date"].values
results["pred_proba_hot"] = y_proba
results["pred_hot"] = y_pred
results["true_hot"] = y_test.values

# Sort by predicted probability descending
results_sorted = results.sort_values("pred_proba_hot", ascending=False)

# Show top 5 predicted hottest stocks
display(results_sorted.head(5))

,regularMarketPrice,regularMarketChangePercent,VolumeSpike,log_marketCap,log_volume,symbol,date,pred_proba_hot,pred_hot,true_hot
14899,18.5200,147.263030,55.681207,21.853444,14.764999,WVE,2025-12-08 21:51:26,1.0,1,1
13822,69.3044,100.533554,14.109827,22.159594,13.866346,GPCR,2025-12-08 17:19:52,1.0,1,1
14599,18.3400,144.763200,54.867382,21.843677,14.764999,WVE,2025-12-08 20:56:27,1.0,1,1
13570,67.6400,95.717580,12.006951,22.135285,13.866346,GPCR,2025-12-08 15:44:13,1.0,1,1
14399,19.4042,158.870820,48.905565,21.900082,14.764999,WVE,2025-12-08 19:51:31,1.0,1,1


In [62]:
import plotly.graph_objects as go
import matplotlib.cm as cm
import matplotlib.colors as colors 
import plotly.express as px

In [73]:
top_symbols = results_sorted.groupby('symbol')['pred_proba_hot'].max().sort_values(ascending=False).head(50).index
df_top = results_sorted[results_sorted['symbol'].isin(top_symbols)]

In [ ]:
heatmap_df = df_top.pivot(index='symbol', columns='date', values='pred_proba_hot')

fig_heatmap = px.imshow(
    heatmap_df,
    labels=dict(x="Snapshot Time", y="Symbol", color="Predicted Hot Probability"),
    color_continuous_scale="YlOrRd",
    text_auto=False,
    aspect="auto"
)

fig_heatmap.update_layout(
    title="Predicted Hot-Stock Probability Heatmap",
    xaxis_nticks=20,
    yaxis={'categoryorder':'total ascending'},
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white'
)

chart_path = os.path.join(OUTPUT_DIR, f"predicted_hot_probability.html")
fig_heatmap.write_html(chart_path, include_plotlyjs='cdn')


In [ ]:
fig_bubble = px.scatter(
    df_top.head(100),  # top 100 predicted hot stocks
    x="VolumeSpike",
    y="regularMarketChangePercent",
    size="log_volume",
    color="pred_proba_hot",
    hover_name="symbol",
    hover_data=["date", "regularMarketPrice", "log_marketCap"],
    color_continuous_scale="YlOrRd",
    size_max=25
)

fig_bubble.update_layout(
    title="Hot Stocks Feature Space",
    xaxis_title="Volume Spike",
    yaxis_title="Price Change %",
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white'
)

chart_path = os.path.join(OUTPUT_DIR, f"hot_stocks_feature_space.html")
fig_bubble.write_html(chart_path, include_plotlyjs='cdn')

In [ ]:
top20 = results_sorted.groupby('symbol')['pred_proba_hot'].max().sort_values(ascending=False).head(20).reset_index()
fig_bar = px.bar(
    top20,
    x='symbol',
    y='pred_proba_hot',
    color='pred_proba_hot',
    color_continuous_scale='YlOrRd',
    text='pred_proba_hot'
)

fig_bar.update_layout(
    title="Top 20 Predicted Hot Stocks",
    xaxis_title="Symbol",
    yaxis_title="Predicted Probability",
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white'
)

chart_path = os.path.join(OUTPUT_DIR, f"top20_predicted_hot_stocks.html")
fig_bar.write_html(chart_path, include_plotlyjs='cdn')